In [2]:
# Install required libraries (run this first)
!pip install -q librosa resampy soundfile numpy pandas scikit-learn matplotlib tensorflow tqdm
print("✅ Installed audio + ML libraries")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.3 MB/s eta 0:00:00
✅ Installed audio + ML libraries


In [7]:
import os
import glob
import shutil
import zipfile
from tqdm import tqdm
import numpy as np
import pandas as pd
import librosa

# For modeling
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [8]:
# Create dataset folders
os.makedirs("/content/dataset/calm", exist_ok=True)
os.makedirs("/content/dataset/angry", exist_ok=True)
os.makedirs("/content/dataset/fearful", exist_ok=True)

print("Dataset folders ensured at /content/dataset/{calm,angry,fearful}")


Dataset folders ensured at /content/dataset/{calm,angry,fearful}


In [10]:
zip_path = "/content/Audio_Song_Actors_01-24.zip"
if os.path.exists(zip_path):
    extract_to = "/content/RAVDESS"
    os.makedirs(extract_to, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_to)
    print("✅ Unzipped RAVDESS to /content/RAVDESS")
else:
    print("No RAVDESS zip file found at /content. Skipping unzip.")


No RAVDESS zip file found at /content. Skipping unzip.


In [15]:
import shutil, glob, os

# Create target folders if not exist
for folder in ["calm", "angry", "fearful"]:
    os.makedirs(f"/content/dataset/{folder}", exist_ok=True)

moved = 0
for file in glob.glob("/content/RAVDESS/**/*.wav", recursive=True):
    fname = os.path.basename(file)
    parts = fname.split("-")

    if len(parts) >= 3:
        code = parts[2]
        target_folder = None

        if code == "02":
            target_folder = "calm"
        elif code == "05":
            target_folder = "angry"
        elif code == "06":
            target_folder = "fearful"

        if target_folder:
            dest_path = f"/content/dataset/{target_folder}/{fname}"
            # Skip if already exists
            if not os.path.exists(dest_path):
                shutil.copy(file, dest_path)
                moved += 1

print(f"✅ Copied {moved} new files (duplicates skipped safely).")

# Display final count in each folder
for folder in ["calm", "angry", "fearful"]:
    count = len(os.listdir(f"/content/dataset/{folder}"))
    print(f"{folder}: {count} files")


✅ Copied 0 new files (duplicates skipped safely).
calm: 8 files
angry: 8 files
fearful: 8 files


In [16]:
# Define feature extraction - consistent across train/predict
def extract_features(file_path, n_mfcc=40, n_chroma=12, n_mels=40):
    try:
        # librosa.load uses resampy internally; installed above.
        y, sr = librosa.load(file_path, sr=None)  # sr=None preserves original sampling rate
        # MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        # Chroma
        stft = np.abs(librosa.stft(y)) if len(y) > 1 else np.abs(np.array([y]))
        chroma = librosa.feature.chroma_stft(S=stft, sr=sr, n_chroma=n_chroma)
        chroma_mean = np.mean(chroma.T, axis=0)
        # Mel Spectrogram
        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        mel_mean = np.mean(mel.T, axis=0)
        # Combine into one vector
        feature_vector = np.hstack([mfccs_mean, chroma_mean, mel_mean])
        return feature_vector
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [17]:
# Extract features from files in the three folders
features = []
labels = []
label_map = {'calm':0, 'angry':1, 'fearful':2}

for emotion in ['calm','angry','fearful']:
    folder = f"/content/dataset/{emotion}"
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith('.wav')])
    print(f"Processing {len(files)} files in {emotion}...")
    for f in files:
        fp = os.path.join(folder, f)
        fv = extract_features(fp)
        if fv is not None:
            features.append(fv)
            labels.append(label_map[emotion])

# Convert to arrays
X = np.array(features)
y = np.array(labels)
print("Extraction done. Feature shape:", X.shape)

# Save to CSV for reuse
if X.size > 0:
    df = pd.DataFrame(X)
    df['label'] = y
    csv_path = "/content/features.csv"
    df.to_csv(csv_path, index=False)
    print(f"✅ features.csv saved to {csv_path} (rows: {df.shape[0]}, cols: {df.shape[1]})")
else:
    print("⚠️ No features extracted. Check your dataset folders; they might be empty.")


Processing 8 files in calm...
Processing 8 files in angry...
Processing 8 files in fearful...
Extraction done. Feature shape: (24, 92)
✅ features.csv saved to /content/features.csv (rows: 24, cols: 93)


In [18]:
# If features.csv exists, load it (otherwise use X,y from above)
csv_path = "/content/features.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    X = df.drop('label', axis=1).values
    y = df['label'].values
    print("Loaded features.csv")
else:
    print("No features.csv found; using in-memory extracted arrays.")

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler for later use
import joblib
joblib.dump(scaler, "/content/scaler.save")

# Train/test split (stratify to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


Loaded features.csv
Train samples: 19, Test samples: 5


In [19]:
num_features = X_train.shape[1]
num_classes = 3

model = Sequential([
    Dense(256, activation='relu', input_shape=(num_features,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=16)
# Save model
model.save('/content/stress_emotion_model.h5')
print("✅ Model trained and saved to /content/stress_emotion_model.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        23,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,155 (254.51 KB)

 Trainable params: 65,155 (254.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 232ms/step - accuracy: 0.2029 - loss: 1.2335 - val_accuracy: 0.6000 - val_loss: 0.9178
Epoch 2/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6502 - loss: 0.8477 - val_accuracy: 0.8000 - val_loss: 0.7898
Epoch 3/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.5175 - loss: 0.8291 - val_accuracy: 0.8000 - val_loss: 0.6913
Epoch 4/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8531 - loss: 0.5085 - val_accuracy: 0.8000 - val_loss: 0.5982
Epoch 5/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9090 - loss: 0.4847 - val_accuracy: 1.0000 - val_loss: 0.5085
Epoch 6/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8882 - loss: 0.4502 - val_accuracy: 1.0000 - val_loss: 0.4345
Epoch 7/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9441 - loss: 0.3043 - val_accuracy: 1.0000 - val_loss: 0.3703
Epoch 8/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8882 - loss: 0.3344 - val_accuracy: 1.0000 - val_loss: 0.3195

✅ Model trained and saved to /content/stress_emotion_model.h5


In [20]:
# Evaluate and show classification report
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {acc*100:.2f}%   Test loss: {loss:.4f}")

# Predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=['calm','angry','fearful']))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))


Test accuracy: 100.00%   Test loss: 0.0287
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step

Classification report:
              precision    recall  f1-score   support

        calm       1.00      1.00      1.00         1
       angry       1.00      1.00      1.00         2
     fearful       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

Confusion matrix:
[[1 0 0]
 [0 2 0]
 [0 0 2]]


In [24]:
# This cell provides a simple file-upload UI and prediction output
from google.colab import files
import joblib
from IPython.display import display, Markdown

uploaded = files.upload()
if len(uploaded) == 0:
    print("No file uploaded.")
else:
    # take the first uploaded file
    fname = list(uploaded.keys())[0]
    print("Uploaded:", fname)
    # Extract features for this file
    fv = extract_features(fname)
    if fv is None:
        print("Feature extraction failed for the uploaded file.")
    else:
        # Load scaler and model (if not in memory)
        scaler = joblib.load("/content/scaler.save")
        model = load_model("/content/stress_emotion_model.h5")
        fv_scaled = scaler.transform(fv.reshape(1, -1))
        probs = model.predict(fv_scaled)[0]  # softmax probabilities
        labels = ['calm','angry','fearful']
        top_idx = np.argmax(probs)
        top_label = labels[top_idx]
        top_conf = probs[top_idx]
        # Binary stressed/not: angry or fearful -> Stressed
        stress_map = { 'calm': 'Not Stressed', 'angry': 'Stressed', 'fearful': 'Stressed' }
        stress_decision = stress_map[top_label]
        # Print nicely
        display(Markdown(f"### Prediction result for **{fname}**"))
        display(Markdown(f"**Predicted Emotion:** `{top_label}`  \n**Confidence:** `{top_conf*100:.2f}%`  \n**Stress Verdict:** `{stress_decision}`"))
        # Also display full probabilities
        prob_text = "\n".join([f"{labels[i]}: {probs[i]*100:.2f}%" for i in range(len(labels))])
        display(Markdown(f"**All class probabilities:**\n{prob_text}"))


Saving 03-02-02-01-01-02-13.wav to 03-02-02-01-01-02-13.wav
Uploaded: 03-02-02-01-01-02-13.wav


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


### Prediction result for **03-02-02-01-01-02-13.wav**

**Predicted Emotion:** `fearful`  
**Confidence:** `100.00%`  
**Stress Verdict:** `Stressed`

**All class probabilities:**
calm: 0.00%
angry: 0.00%
fearful: 100.00%